In [1]:
import pandas as pd
import pathlib
from tabulate import tabulate
import sys
from pathlib import Path

project_path = Path().resolve().parent.parent
datapath = project_path / "data"

if str(project_path) not in sys.path:
    sys.path.append(str(project_path))

In [2]:
# read all csv files in the data folder
dfs = {}
print("Reading csv files, data file name and shape:")
for f in datapath.glob("*.csv"):
    df = pd.read_csv(f)
    print(f"\t- {f.stem}: {df.shape}")
    dfs[f.stem] = df

Reading csv files, data file name and shape:
	- sample_submission: (25, 4)
	- test_series_descriptions: (3, 3)
	- train: (1975, 26)
	- train_label_coordinates: (48692, 7)
	- train_series_descriptions: (6294, 3)


## `train.csv`
- There are 1975 rows and 26 columns.
- Each row represents a unique `study_id`.
- There are 5 diffrent conditions in the dataset <br>
  - spinal_canal_stenosis
  - right_subarticular_stenosis
  - left_subarticular_stenosis
  - left_neural_foraminal_narrowing
  - right_neural_foraminal_narrowing
- Each condition has 5 different levels
  - l1_l2
  - l2_l3
  - l3_l4
  - l4_l5
  - l5_s1
- Distribution of the conditions is not balanced
  

In [3]:
train_df = dfs["train"].copy()
print(f"Train data shape: {train_df.shape}")
train_df.head()

Train data shape: (1975, 26)


,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
0,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
1,4646740,Normal/Mild,Normal/Mild,Moderate,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Moderate,Moderate,Moderate,Normal/Mild
2,7143189,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
3,8785691,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
4,10728036,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild


In [4]:
train_df.shape[0], train_df["study_id"].nunique()

(1975, 1975)

In [32]:
conditions = [c[:-6] for c in train_df.columns[1:]]
levels = [c[-5:] for c in train_df.columns[1:]]

def unique_unsorted(lst):
    seen = set()
    return [x for x in lst if not (x in seen or seen.add(x))]

# Get unique elements while preserving order
conditions = unique_unsorted(conditions)
levels = unique_unsorted(levels)
print(conditions)
print(levels)

['spinal_canal_stenosis', 'left_neural_foraminal_narrowing', 'right_neural_foraminal_narrowing', 'left_subarticular_stenosis', 'right_subarticular_stenosis']
['l1_l2', 'l2_l3', 'l3_l4', 'l4_l5', 'l5_s1']


In [6]:
group_cols = {c: 0 for c in conditions}
for col in train_df.columns[1:]:
    col_short = col[:-6]
    if col_short in conditions:
        group_cols[col_short] += 1
print("Number of columns for each condition:")
group_cols

Number of columns for each condition:


{'left_neural_foraminal_narrowing': 5,
 'right_subarticular_stenosis': 5,
 'left_subarticular_stenosis': 5,
 'spinal_canal_stenosis': 5,
 'right_neural_foraminal_narrowing': 5}

In [61]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Calculate the counts for each condition in each column
severities = ['Normal/Mild', 'Moderate', 'Severe']

columns = train_df.columns[1:]  # Exclude the 'study_id' column

counts = {severity: [] for severity in severities}

for column in columns:
    column_counts = train_df[column].value_counts()
    for severity in severities:
        counts[severity].append(column_counts.get(severity, 0))

# Define a color palette
color_palette = {'Normal/Mild': '#66c2a5', 'Moderate': '#fc8d62', 'Severe': '#8da0cb'}

# Create subplots
fig = make_subplots(rows=2, cols=3, subplot_titles=[f"{conditions[i]}" for i in range(5)] + [""], vertical_spacing=0.12)

# Add traces to subplots
for i in range(5):  # Only iterate through 5 subplots
    row = i // 3 + 1
    col = i % 3 + 1
    start_idx = i * 5
    end_idx = start_idx + 5
    
    # Get the last 5 characters of each column name
    short_labels = [col[-5:] for col in columns[start_idx:end_idx]]
    
    for severity in severities:
        fig.add_trace(
            go.Bar(
                x=short_labels,  # Use the shortened labels here
                y=counts[severity][start_idx:end_idx],
                name=severity,
                marker_color=color_palette[severity],
                showlegend=True if i == 0 else False  # Show legend only for the first subplot
            ),
            row=row, col=col
        )

# Update layout
fig.update_layout(
    barmode='stack',
    title='<b>Distribution of Conditions-Levels-Severities</b>',
    title_x=0.5,
    title_font_size=30,
    height=800,  # Reduced height to bring rows closer
    width=1500,
    showlegend=True,
    legend=dict(orientation="v", yanchor="bottom", y=0.3, xanchor="right", x=0.9, font=dict(size=20)),
    font=dict(size=15)  # Increase overall font size
)
fig.update_xaxes(showticklabels=False, showline=False, title=None, row=2, col=3)
fig.update_yaxes(showticklabels=False, showline=False, title=None, row=2, col=3)

fig.show()

## `train_series_descriptions.csv`
- There are 1975 `study_id`, same as `train.csv`.
- 3 different `series_description`: Axial T2, Sagittal T2/STIR, Sagittal T1
- Some `study_id` has more `series_description`
  - The most: 4096820034 has 6
  - The least: 2780132468 has 2
- `series_id`

In [62]:
df = dfs["train_series_descriptions"].copy()
print(f"train_series_descriptions shape: {df.shape}")
df.head()

train_series_descriptions shape: (6294, 3)


,study_id,series_id,series_description
0,4003253,702807833,Sagittal T2/STIR
1,4003253,1054713880,Sagittal T1
2,4003253,2448190387,Axial T2
3,4646740,3201256954,Axial T2
4,4646740,3486248476,Sagittal T1


In [63]:
# All study_id in train_series_descriptions.csv are in train.csv
train_study_ids = set(dfs["train"].study_id)
df[
    ~df["study_id"].isin(train_study_ids)
]

,study_id,series_id,series_description


In [64]:
print(f"Number of unique study_id: {df['study_id'].nunique()}\n")
print(f"Value counts {df['study_id'].value_counts()}")

Number of unique study_id: 1975

Value counts study_id
4096820034    6
886995462     5
2109299850    5
2622319181    5
2626030939    5
             ..
1586052047    3
4290709089    3
2492114990    2
3008676218    2
2780132468    2
Name: count, Length: 1975, dtype: int64


In [100]:
df[
    df['study_id'] == 4096820034
]

,study_id,series_id,series_description
5997,4096820034,300517765,Axial T2
5998,4096820034,2097107888,Axial T2
5999,4096820034,2602265508,Sagittal T2/STIR
6000,4096820034,2679683906,Axial T2
6001,4096820034,3114813181,Axial T2
6002,4096820034,3236751045,Sagittal T1


In [113]:
df[
    df['study_id'] == 2780132468
]

,study_id,series_id,series_description
4056,2780132468,1848483560,Sagittal T2/STIR
4057,2780132468,4151611107,Axial T2


In [114]:
print(f"Value counts {df['series_description'].value_counts()}")

Value counts series_description
Axial T2            2340
Sagittal T1         1980
Sagittal T2/STIR    1974
Name: count, dtype: int64


## `train_label_coordinates.csv`

In [116]:
df = dfs["train_label_coordinates"].copy()
print(f"train_label_coordinates shape: {df.shape}")
df.head()

train_label_coordinates shape: (48692, 7)


,study_id,series_id,instance_number,condition,level,x,y
0,4003253,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602
1,4003253,702807833,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286
2,4003253,702807833,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182
3,4003253,702807833,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434
4,4003253,702807833,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602


In [152]:
print(f"Number of unique study_id: {df['study_id'].nunique()}\n")

Number of unique study_id: 1974



In [150]:
l1 = df['study_id'].unique()
l2 = dfs["train"]['study_id'].unique()
set(l2) - set(l1)

{3008676218}

In [146]:
train_study_ids = set(dfs["train"].study_id)
train_study_ids_2 = set(dfs["train_label_coordinates"].study_id)
# all id that are in study_id but not in study_id_2
train_study_ids - train_study_ids_2

{3008676218}

In [147]:
_df = dfs['train']
_df[_df['study_id'] == 3008676218]

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
1378,3008676218,NaN,NaN,NaN,NaN,NaN,Normal/Mild,Severe,Moderate,Moderate,...,Normal/Mild,Moderate,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Severe,Normal/Mild,Normal/Mild


In [139]:
train_study_ids == train_study_ids_2

True

In [131]:
len(train_study_ids_2)

1975